# Ayudantía: análisis de datos
##### Por: Daniela Flores Villanueva

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import datetime
from sklearn import linear_model
from sklearn import metrics
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Los datos que utilizaremos en esta ayudantía corresponden al *data set* [Top Spotify Tracks of 2017](https://www.kaggle.com/nadintamer/top-tracks-of-2017), al que se le agregó la columna *listeners*, mediante consultas a la API de LastFM. En él, cada fila del archivo representa una canción, descrita por las *features* que se pueden encontrar [aquí](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/).

In [ ]:
top_tracks = pd.read_csv("top_tracks_2017.csv")

## Exploración de los datos

Un primer acercamiento a los datos consiste en revisar cómo lucen:

In [ ]:
top_tracks.head()

In [ ]:
top_tracks.describe()

¿Cuáles son las canciones top 10 según cantidad de oyentes?

In [ ]:
top_tracks.sort_values(by="listeners", ascending=False).head(10)

Veamos si existen datos nulos:

In [ ]:
top_tracks.apply(lambda x: sum(x.isnull()), axis=0)

La siguiente consulta en `pandas` nos permite saber con facilidad cuál es la canción que tiene una cantidad inválida de oyentes.

In [ ]:
top_tracks[top_tracks["listeners"].isnull()]

Procedemos a eliminar esa fila.

In [ ]:
top_tracks.dropna(inplace=True)

Ahora bien, ¿habrá alguna fila en el *data set* que contenga algún dato *sin sentido*? Para el caso particular de estos datos, esto se traduciría en una candidad de oyentes igual a 0.

In [ ]:
top_tracks[top_tracks["listeners"] == 0]

Eliminamos también esa fila, a través de la siguiente consulta:

In [ ]:
top_tracks = top_tracks[top_tracks["listeners"] != 0]

¿Compartirán las canciones top algunas características? Averigüémoslo con los siguientes gráficos:

In [ ]:
top_tracks["danceability"].plot(kind="hist", bins=5, edgecolor="black", grid=False, title="Danceability histogram")

In [ ]:
top_tracks["energy"].plot(kind="hist", bins=5, edgecolor="black", grid=False, title="Energy histogram")

In [ ]:
top_tracks["duration_ms"].plot(kind="hist", bins=5, edgecolor="black", grid=False, title="Duration histogram")

¿Cuál es el intervalo de duración de la mayoría de las canciones en el top 100 2017? En un formato más humanamente legible:

In [ ]:
print(datetime.timedelta(milliseconds=200000))
print(datetime.timedelta(milliseconds=262500))

In [ ]:
top_tracks["acousticness"].plot(kind="hist", bins=5, edgecolor="black", grid=False, title="Acousticness histogram")

`pandas` nos permite revisar la correlación entre todas las variables del *data set*. Para esto, utilizaremos solo las columnas cuyos valores están entre 0 y 1 (de acuerdo a lo expuesto en la documentación de Spotify).

In [ ]:
normalized_columns = ["danceability", "energy", "speechiness", "acousticness", "instrumentalness",
                     "liveness", "valence"]

La matriz de correlación es la siguiente:

In [ ]:
top_tracks[normalized_columns].corr()

En forma gráfica:

In [ ]:
plt.matshow(top_tracks[normalized_columns].corr())
plt.show()

¿Cuáles serán las características más distintivas de las canciones top 2017?

In [ ]:
top_tracks[normalized_columns].mean().plot(kind='bar')

El gráfico nos permite concluir que las canciones más populares son bailables, energéticas y tienen una alta valencia.

# Regresión

Ahora, introduciremos un *data set* correspondiente a algunas canciones escuchadas por cierta ayudante del curso. Estos datos fueron obtenidos gracias a [este *script*](https://github.com/juandes/spotify-audio-features-data-experiment/blob/master/get_data.py).

In [ ]:
test_dataset = pd.read_csv("danielaflores777-1.csv")
test_dataset.head()

Veamos las similitudes y diferencias entre ambos *data sets* en forma gráfica.

In [ ]:
plt.bar(normalized_columns, top_tracks[normalized_columns].mean(), color="#a4b8c4")
plt.bar(normalized_columns, test_dataset[normalized_columns].mean(), color="#0cca4a", alpha= 0.5)
plt.legend(["top", "ayudante"])
plt.title("Comparación entre los data sets")
plt.xlabel("Audio feature")
plt.ylabel("Value")
plt.xticks(rotation='vertical')

Intentemos ahora predecir la valencia de las canciones oídas por su ayudante a partir de la *danzabilidad*, mediante **regresión lineal**. Con este objetivo en mente, guardamos en `train_dataset` las filas de `top_tracks`, pero solo con las columnas normalizadas.

In [ ]:
train_dataset = top_tracks[normalized_columns]
train_dataset.head()

Ahora, en `X_train` dejaremos la variable explicativa (*danceability*) y en `y_train` el regresando (*valence*). Ocurrirá algo similar para el set de prueba.

In [ ]:
X_train = train_dataset["danceability"].values.reshape(-1, 1)
y_train = train_dataset["valence"]
X_test = test_dataset["danceability"].values.reshape(-1, 1)
y_test = test_dataset["valence"]

Procedemos a ajustar la regresión lineal.

In [ ]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

Veamos el desempeño de nuestro modelo a través del coeficiente de determinación ($R^2$).

In [ ]:
model.score(X_test, y_test)

De acuerdo a la [documentación](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.score):
> The coefficient R^2 is defined as (1 - u/v), where u is the residual sum of squares ((y_true - y_pred) ^ 2).sum() and v is the total sum of squares ((y_true - y_true.mean()) ^ 2).sum(). The best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected value of y, disregarding the input features, would get a R^2 score of 0.0.

Grafiquemos los datos y la recta de regresión:

In [ ]:
plt.scatter(X_test, y_test,  color='black')
plt.plot(X_test, predictions, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())
plt.title("Danceability and Valence")
plt.xlabel("Danceability")
plt.ylabel("Valence")

## Extra: breve análisis de los hábitos musicales de una persona real

In [ ]:
dataframe = pd.read_csv("datos_reales_usuario.csv")

In [ ]:
dataframe.head()

Convirtamos las fechas a nuestra zona horaria (sabemos de antemano que vive en Chile):

In [ ]:
dataframe["Date"] = pd.to_datetime(dataframe["Date"])
dataframe["Date"] = dataframe["Date"].dt.tz_localize('UTC').dt.tz_convert("Chile/Continental")

¿Cuáles son las 10 canciones más escuchadas?

In [ ]:
top_10_songs = dataframe.groupby(["Track", "Artist"]).size().reset_index(name='counts')
top_10_songs = top_10_songs.sort_values(by="counts", ascending=False).iloc[0:10]
top_10_songs

¿Cuál es la evolución de la cantidad de canciones escuchadas por mes?

In [ ]:
new = dataframe.set_index("Date")["Track"]
new.index = pd.to_datetime(new.index)

In [ ]:
months = new.resample("M", convention="start").count()
months.plot()
plt.title("Evolución de la cantidad de canciones escuchadas por mes")
plt.show()

¿Cuáles son las horas más activas de este usuario?

In [ ]:
hours = new.groupby(new.index.hour).count()
hours = pd.DataFrame(hours)
plt.bar(x=hours.index, height=hours["Track"])
plt.title("Canciones escuchadas según la hora")
plt.xticks(range(0,24))
plt.show()